In [1]:
import tensorflow as tf
from tensorflow.keras import layers
import utils
from tensorflow.keras import mixed_precision
import os
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam, SGD
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
mixed_precision.set_global_policy('mixed_float16')
import logging
tf.get_logger().setLevel(logging.ERROR)

2024-07-25 19:31:06.902335: I tensorflow/core/util/port.cc:111] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-07-25 19:31:06.927168: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-07-25 19:31:06.927200: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-07-25 19:31:06.927225: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
2024-07-25 19:31:06.932256: I tensorflow/core/platform/cpu_feature_g

INFO:tensorflow:Mixed precision compatibility check (mixed_float16): OK
Your GPU will likely run quickly with dtype policy mixed_float16 as it has compute capability of at least 7.0. Your GPU: NVIDIA GeForce RTX 4070 Laptop GPU, compute capability 8.9


2024-07-25 19:31:08.600611: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-25 19:31:08.629022: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-25 19:31:08.631545: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

In [2]:
"""
#Batching using prefetch
train_data_casted = train_data.map(map_func = preprocess_image, num_parallel_calls = tf.data.AUTOTUNE).shuffle(buffer_size = 1000).batch(batch_size = 32).prefetch(buffer_size = tf.data.AUTOTUNE)
test_data_casted = test_data.map(map_func = preprocess_image, num_parallel_calls = tf.data.AUTOTUNE).batch(32).prefetch(tf.data.AUTOTUNE)
"""

'\n#Batching using prefetch\ntrain_data_casted = train_data.map(map_func = preprocess_image, num_parallel_calls = tf.data.AUTOTUNE).shuffle(buffer_size = 1000).batch(batch_size = 32).prefetch(buffer_size = tf.data.AUTOTUNE)\ntest_data_casted = test_data.map(map_func = preprocess_image, num_parallel_calls = tf.data.AUTOTUNE).batch(32).prefetch(tf.data.AUTOTUNE)\n'

In [3]:
fundus_train = "/home/thefilthysalad/PycharmProjects/eye_detection_fundus_dataset/Dataset/split1/train"
fundus_test = "/home/thefilthysalad/PycharmProjects/eye_detection_fundus_dataset/Dataset/split1/test"


BATCH_SIZE = 32
IMG_SIZE = (224, 224)


In [4]:
print(os.listdir(fundus_train))

['glaucoma', 'normal', 'cataract', 'diabetic_retinopathy']


In [5]:
IMG_HEIGHT, IMG_WIDTH = 224, 224

In [6]:
train_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    fundus_train,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    shuffle=True,
    seed=123,
    validation_split=0.3,
    subset='training'
)

validation_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    fundus_train,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    shuffle=False,
    seed=123,
    validation_split=0.3,
    subset='validation'
)
test_dataset = tf.keras.preprocessing.image_dataset_from_directory(
    fundus_test,
    labels='inferred',
    label_mode='categorical',
    batch_size=BATCH_SIZE,
    image_size=(IMG_HEIGHT, IMG_WIDTH),
    shuffle=False,
    seed=123,

)

Found 3372 files belonging to 4 classes.
Using 2361 files for training.


2024-07-25 19:31:09.810645: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-25 19:31:09.813129: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysfs-bus-pci#L344-L355
2024-07-25 19:31:09.815547: I tensorflow/compiler/xla/stream_executor/cuda/cuda_gpu_executor.cc:894] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero. See more at https://github.com/torvalds/linux/blob/v6.0/Documentation/ABI/testing/sysf

Found 3372 files belonging to 4 classes.
Using 1011 files for validation.
Found 845 files belonging to 4 classes.


In [7]:
train_dataset

<_PrefetchDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 4), dtype=tf.float32, name=None))>

In [8]:
#Using tf prefetch dataset
preprocess_input = tf.keras.applications.resnet.preprocess_input

In [9]:
train_datagen = train_dataset.map(lambda x, y: (preprocess_input(x), y))
val_datagen = validation_dataset.map(lambda x, y: (preprocess_input(x), y))
test_datagen = test_dataset.map(lambda x, y: (preprocess_input(x), y))

In [10]:
train_datagen

<_MapDataset element_spec=(TensorSpec(shape=(None, 224, 224, 3), dtype=tf.float32, name=None), TensorSpec(shape=(None, 4), dtype=tf.float32, name=None))>

In [11]:
import tensorflow.keras.applications as apps
IMG_WIDTH, IMG_HEIGHT = 224, 224
base_model = apps.ResNet152(weights = 'imagenet', include_top = False, input_shape = (IMG_WIDTH, IMG_HEIGHT, 3))
base_model.trainable = False

In [12]:

for i in base_model.layers:
    print(f'Layer: {i.name}, {i.trainable}')

Layer: input_1, False
Layer: conv1_pad, False
Layer: conv1_conv, False
Layer: conv1_bn, False
Layer: conv1_relu, False
Layer: pool1_pad, False
Layer: pool1_pool, False
Layer: conv2_block1_1_conv, False
Layer: conv2_block1_1_bn, False
Layer: conv2_block1_1_relu, False
Layer: conv2_block1_2_conv, False
Layer: conv2_block1_2_bn, False
Layer: conv2_block1_2_relu, False
Layer: conv2_block1_0_conv, False
Layer: conv2_block1_3_conv, False
Layer: conv2_block1_0_bn, False
Layer: conv2_block1_3_bn, False
Layer: conv2_block1_add, False
Layer: conv2_block1_out, False
Layer: conv2_block2_1_conv, False
Layer: conv2_block2_1_bn, False
Layer: conv2_block2_1_relu, False
Layer: conv2_block2_2_conv, False
Layer: conv2_block2_2_bn, False
Layer: conv2_block2_2_relu, False
Layer: conv2_block2_3_conv, False
Layer: conv2_block2_3_bn, False
Layer: conv2_block2_add, False
Layer: conv2_block2_out, False
Layer: conv2_block3_1_conv, False
Layer: conv2_block3_1_bn, False
Layer: conv2_block3_1_relu, False
Layer: con

In [13]:
len(base_model.layers)

515

In [14]:
No_of_classes = len(os.listdir(fundus_train))
No_of_classes

4

In [15]:
aug_layer = utils.return_data_aug_layer_for_eff_net()

In [17]:
from tensorflow.keras.layers import Dense, GlobalAvgPool2D, Input, Dropout
from tensorflow.keras.models import Model
inputs = Input(shape = (IMG_HEIGHT, IMG_WIDTH, 3), name = 'Input_layer')
x = aug_layer(inputs)
#x = tf.keras.layers.Rescaling(1./255)(inputs)
x = base_model(x, training = False)
x = layers.GlobalAvgPool2D()(x)


x = tf.keras.layers.Dense(256, activation='relu')(x)
x = tf.keras.layers.Dense(128, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
x = tf.keras.layers.Dense(64, activation='relu')(x)
x = tf.keras.layers.Dropout(0.5)(x)
Outputs = Dense(No_of_classes, activation = 'softmax', dtype = tf.float32)(x)

model_1 = Model(inputs, Outputs, name = 'Resnet')

model_1.summary()

Model: "Resnet"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 Data_Augmentation (Sequent  (None, None, None, 3)     0         
 ial)                                                            
                                                                 
 resnet152 (Functional)      (None, 7, 7, 2048)        58370944  
                                                                 
 global_average_pooling2d_1  (None, 2048)              0         
  (GlobalAveragePooling2D)                                       
                                                                 
 dense_4 (Dense)             (None, 256)               524544    
                                                                 
 dense_5 (Dense)             (None, 128)               32896

In [18]:
#Model checkpointing
from tensorflow.keras.callbacks import ModelCheckpoint
model_1chkpt = ModelCheckpoint(filepath = os.path.join('Trained_Models',model_1.name), save_weights_only = False, save_best_only = True, verbose = 1)

In [19]:
from keras.src.callbacks import ReduceLROnPlateau

lr_scheduler = ReduceLROnPlateau(factor=0.3, patience=3) 

In [20]:
#model compilation
from tensorflow.keras.losses import categorical_crossentropy
from tensorflow.keras.optimizers import Adam
model_1.compile(loss = 'categorical_crossentropy', optimizer = Adam(learning_rate = 0.001), metrics = ['accuracy'])

In [ ]:
history_1 = model_1.fit(train_datagen, validation_data = (val_datagen), epochs = 10, verbose = 1, callbacks = [model_1chkpt, lr_scheduler], steps_per_epoch = len(train_datagen), validation_steps = len(val_datagen))

Epoch 1/10


In [21]:
base_model.trainable = True

In [22]:
for layer in base_model.layers[:-75]:
    layer.trainable = False

In [23]:
for layer in base_model.layers:
    print(layer.trainable)

False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
Fals

In [25]:
start_epoch = 5
model_1.compile(loss = 'categorical_crossentropy', optimizer = Adam(learning_rate = 0.0003), metrics = ['accuracy'])
model_1.summary()

Model: "Resnet"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 resnet152 (Functional)      (None, 7, 7, 2048)        58370944  
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 256)               524544    
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                            

In [26]:
history_50 = model_1.fit(train_datagen, validation_data = (val_datagen), epochs = start_epoch+5, initial_epoch = start_epoch, verbose = 1, callbacks = [model_1chkpt, lr_scheduler], steps_per_epoch = len(train_datagen), validation_steps = len(val_datagen))

Epoch 6/10


2024-07-25 19:25:41.282059: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 602.62MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


73/74 [============================>.] - ETA: 0s - loss: 1.5054 - accuracy: 0.5890

2024-07-25 19:25:49.173456: W tensorflow/tsl/framework/bfc_allocator.cc:296] Allocator (GPU_0_bfc) ran out of memory trying to allocate 601.29MiB with freed_by_count=0. The caller indicates that this is not a failure, but this may mean that there could be performance gains if more memory were available.


74/74 [==============================] - ETA: 0s - loss: 1.5075 - accuracy: 0.5870
Epoch 6: val_loss did not improve from 0.24127
74/74 [==============================] - 21s 145ms/step - loss: 1.5075 - accuracy: 0.5870 - val_loss: 0.6559 - val_accuracy: 0.8813 - lr: 3.0000e-04
Epoch 7/10
74/74 [==============================] - ETA: 0s - loss: 0.7294 - accuracy: 0.6972
Epoch 7: val_loss did not improve from 0.24127
74/74 [==============================] - 9s 118ms/step - loss: 0.7294 - accuracy: 0.6972 - val_loss: 0.7996 - val_accuracy: 0.5312 - lr: 3.0000e-04
Epoch 8/10
74/74 [==============================] - ETA: 0s - loss: 0.5723 - accuracy: 0.7967
Epoch 8: val_loss did not improve from 0.24127
74/74 [==============================] - 9s 121ms/step - loss: 0.5723 - accuracy: 0.7967 - val_loss: 0.6676 - val_accuracy: 0.8902 - lr: 3.0000e-04
Epoch 9/10
74/74 [==============================] - ETA: 0s - loss: 0.4241 - accuracy: 0.8636
Epoch 9: val_loss did not improve from 0.24127
74

In [27]:
start_epoch = 10
for layer in base_model.layers[-150:]:
    layer.trainable = True

In [28]:
model_1.compile(loss = 'categorical_crossentropy', optimizer = Adam(learning_rate = 0.0001), metrics = ['accuracy'])
model_1.summary()

Model: "Resnet"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 resnet152 (Functional)      (None, 7, 7, 2048)        58370944  
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 256)               524544    
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dropout (Dropout)           (None, 128)               0         
                                                            

In [29]:
history_100 = model_1.fit(train_datagen, validation_data = (val_datagen), epochs = start_epoch+5, initial_epoch = start_epoch, verbose = 1, callbacks = [model_1chkpt, lr_scheduler], steps_per_epoch = len(train_datagen), validation_steps = len(val_datagen))

Epoch 11/15


2024-07-25 19:27:26.288457: W tensorflow/tsl/framework/bfc_allocator.cc:485] Allocator (GPU_0_bfc) ran out of memory trying to allocate 12.25MiB (rounded to 12845056)requested by op Resnet/resnet152/conv4_block35_3_conv/Conv2D
If the cause is memory fragmentation maybe the environment variable 'TF_GPU_ALLOCATOR=cuda_malloc_async' will improve the situation. 
Current allocation summary follows.
Current allocation summary follows.
2024-07-25 19:27:26.288675: I tensorflow/tsl/framework/bfc_allocator.cc:1039] BFCAllocator dump for GPU_0_bfc
2024-07-25 19:27:26.288718: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (256): 	Total Chunks: 214, Chunks in use: 214. 53.5KiB allocated for chunks. 53.5KiB in use in bin. 11.9KiB client-requested in use in bin.
2024-07-25 19:27:26.288736: I tensorflow/tsl/framework/bfc_allocator.cc:1046] Bin (512): 	Total Chunks: 93, Chunks in use: 91. 46.8KiB allocated for chunks. 45.5KiB in use in bin. 45.5KiB client-requested in use in bin.
2024-07-25 19:2

ResourceExhaustedError: Graph execution error:

Detected at node Resnet/resnet152/conv4_block35_3_conv/Conv2D defined at (most recent call last):
  File "/usr/lib/python3.10/runpy.py", line 196, in _run_module_as_main

  File "/usr/lib/python3.10/runpy.py", line 86, in _run_code

  File "/home/thefilthysalad/PycharmProjects/eye_detection_fundus_dataset/.venv/lib/python3.10/site-packages/ipykernel_launcher.py", line 18, in <module>

  File "/home/thefilthysalad/PycharmProjects/eye_detection_fundus_dataset/.venv/lib/python3.10/site-packages/traitlets/config/application.py", line 1075, in launch_instance

  File "/home/thefilthysalad/PycharmProjects/eye_detection_fundus_dataset/.venv/lib/python3.10/site-packages/ipykernel/kernelapp.py", line 739, in start

  File "/home/thefilthysalad/PycharmProjects/eye_detection_fundus_dataset/.venv/lib/python3.10/site-packages/tornado/platform/asyncio.py", line 205, in start

  File "/usr/lib/python3.10/asyncio/base_events.py", line 603, in run_forever

In [29]:
start_epoch = 40
for layer in base_model.layers[-250:]:
    layer.trainable = True

In [30]:
model_1.compile(loss = 'categorical_crossentropy', optimizer = Adam(learning_rate = 0.00009), metrics = ['accuracy'])
model_1.summary()

Model: "MobilenetV3L"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 Input_layer (InputLayer)    [(None, 224, 224, 3)]     0         
                                                                 
 Data_Augmentation (Sequent  (None, None, None, 3)     0         
 ial)                                                            
                                                                 
 MobilenetV3large (Function  (None, 7, 7, 960)         2996352   
 al)                                                             
                                                                 
 global_average_pooling2d (  (None, 960)               0         
 GlobalAveragePooling2D)                                         
                                                                 
 dense (Dense)               (None, 256)               246016    
                                                      

In [31]:
history_200 = model_1.fit(train_datagen, validation_data = (val_datagen), epochs = start_epoch+50, initial_epoch = start_epoch, verbose = 1, callbacks = [model_1chkpt, lr_scheduler], steps_per_epoch = len(train_datagen), validation_steps = len(val_datagen))

Epoch 41/90
74/74 [==============================] - ETA: 0s - loss: 0.3224 - accuracy: 0.8937
Epoch 41: val_loss did not improve from 0.26771
74/74 [==============================] - 42s 421ms/step - loss: 0.3224 - accuracy: 0.8937 - val_loss: 0.4803 - val_accuracy: 0.8694 - lr: 9.0000e-05
Epoch 42/90
74/74 [==============================] - ETA: 0s - loss: 0.2550 - accuracy: 0.9119
Epoch 42: val_loss did not improve from 0.26771
74/74 [==============================] - 23s 306ms/step - loss: 0.2550 - accuracy: 0.9119 - val_loss: 0.7618 - val_accuracy: 0.7201 - lr: 9.0000e-05
Epoch 43/90
74/74 [==============================] - ETA: 0s - loss: 0.2707 - accuracy: 0.9111
Epoch 43: val_loss did not improve from 0.26771
74/74 [==============================] - 20s 272ms/step - loss: 0.2707 - accuracy: 0.9111 - val_loss: 0.3554 - val_accuracy: 0.8912 - lr: 9.0000e-05
Epoch 44/90
74/74 [==============================] - ETA: 0s - loss: 0.2477 - accuracy: 0.9123
Epoch 44: val_loss did not im

KeyboardInterrupt: 

In [11]:
resnet = tf.keras.models.load_model("/home/thefilthysalad/PycharmProjects/eye_detection_fundus_dataset/ML_Models/Trained_Models/Resnet_88")


In [12]:
resnet.evaluate(test_datagen)

2024-07-25 19:28:48.886377: I tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:442] Loaded cuDNN version 8600
2024-07-25 19:28:49.001810: I tensorflow/tsl/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


27/27 [==============================] - 4s 84ms/step - loss: 0.3202 - accuracy: 0.8793


[0.3201874792575836, 0.8792899250984192]